# Demo: Defining Control_M Workflows using Python

# Step 1 - Setup

## Step 1A - Install the library

In [1]:
!pip --version

pip 21.2.4 from /opt/conda/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/tadinve/naga.git

  Cloning https://github.com/tadinve/naga.git to /tmp/pip-req-build-ig3twqti
  Running command git clone -q https://github.com/tadinve/naga.git /tmp/pip-req-build-ig3twqti
  Resolved https://github.com/tadinve/naga.git to commit 2c2729372cb122df0db36c2da19df68bf6384a7b
  Created wheel for ctm-python-client: filename=ctm_python_client-0.3.0-py3-none-any.whl size=615428 sha256=baf1f606033fe36887f45be273d627daa39dda02588b040d5c874c9d99c8b477
  Stored in directory: /tmp/pip-ephem-wheel-cache-icym8rqe/wheels/de/35/cb/b70c2ffd6e40942c62635105854bc5fabd5125a0f0f9b17cb9
Successfully built ctm-python-client
  Attempting uninstall: ctm-python-client
    Found existing installation: ctm-python-client 0.3.0
    Uninstalling ctm-python-client-0.3.0:
      Successfully uninstalled ctm-python-client-0.3.0


In [3]:
from ctm_python_client.core.bmc_control_m import CmJobFlow
from ctm_python_client.jobs.dummy import DummyJob

# Step 2 - Instantiate, Authenticate and Schedule


## Step 2A - Create the object

In [4]:
# Please change the URfrI, and ctm_user and enter ctm_password to match your environment
from ctm_python_client.session.session import Session

import getpass

ctm_uri = "https://acb-rhctmv20.centralus.cloudapp.azure.com:8443/automation-api"
ctm_user = "vtadinad"
ctm_pwd = "P4ssw0rd"
if "ctm_pwd" not in locals():  # has not been enterd once, will skip next time
    ctm_pwd = getpass.getpass("Enter your Control M Password ")

session = Session(endpoint=ctm_uri, username=ctm_user, password=ctm_pwd)
session.get_token()

'C58DAD3C140CF0A58041B4FF655316A06AB6646EC4C5813EA84926A8BCC04F18740DF2A5046147104A3C9395A58E2CC9E23DE3C9402974C734526345B9F2EB38'

In [5]:
t1_flow = CmJobFlow(
    application="Naga0.3_Examples", sub_application="Demo-OR_JOB", session=session
)

## Step 2B - Define the Schedule

In [6]:
t1_flow.set_run_as(username="ctmuser", host="acb-rhctmv20")

In [7]:
# Define the schedule
months = ["JAN", "OCT", "DEC"]
monthDays = ["ALL"]
weekDays = ["MON", "TUE", "WED", "THU", "FRI"]
fromTime = "0300"
toTime = "2100"
t1_flow.set_schedule(months, monthDays, weekDays, fromTime, toTime)

# Step 3  - Create Folder

In [10]:
# Create Fodler
f1 = t1_flow.create_folder(name="OR-JOB")

# Step 4 - Create Tasks

In [11]:
start = t1_flow.add_job(f1, DummyJob(f1, "Start-Flow"))


job1 = DummyJob(f1, "Job1")
job1.add_if_output("if-true", "*true*", "Job1-TO-Job2")
job1.add_if_output("if-flase", "*false*", "Job1-TO-Job3")
job1_id = t1_flow.add_job(f1, job1)

job2 = DummyJob(f1, "Job2")
job2_id = t1_flow.add_job(f1, job2)

job3 = DummyJob(f1, "Job3")
job3_id = t1_flow.add_job(f1, job3)

job4 = DummyJob(f1, "Job4")
job4.wait_for_jobs(job1.get_job_name(), job3.get_job_name(), condition="OR")
job4_id = t1_flow.add_job(f1, job4)

end = t1_flow.add_job(f1, DummyJob(f1, "End-Flow"))

# Step 5 - Chain Tasks

In [12]:
# start >>  hello_world_id >> end
t1_flow.chain_jobs(f1, [start, job1_id])
t1_flow.chain_jobs(f1, [job2_id, job3_id])
t1_flow.chain_jobs(f1, [job4_id, end])

[('Start-Flow', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfcec910>), ('Job1', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfcec490>), ('Job2', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfcec5b0>), ('Job3', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfcec5e0>), ('Start-Flow', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfc84eb0>), ('Job1', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfc84160>), ('Job2', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfc84f10>), ('Job3', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfc842b0>), ('Job4', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfc84910>), ('End-Flow', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfc848b0>)]
['4', '5']
[('Start-Flow', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfcec910>), ('Job1', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f55dfcec490>), ('Job2', <ctm_python_client.jobs.dummy.DummyJob objec

# Step 6 - Display Workflow

## Step 6A - Display DAG

In [13]:
# View the t1_flow Details
nodes, edges = t1_flow.get_nodes_and_edges()
nodes, edges

([('Start-Flow', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfcec910>),
  ('Job1', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfcec490>),
  ('Job2', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfcec5b0>),
  ('Job3', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfcec5e0>),
  ('Start-Flow', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfc84eb0>),
  ('Job1', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfc84160>),
  ('Job2', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfc84f10>),
  ('Job3', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfc842b0>),
  ('Job4', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfc84910>),
  ('End-Flow', <ctm_python_client.jobs.dummy.DummyJob at 0x7f55dfc848b0>)],
 [('Start-Flow', 'Job1'), ('Job2', 'Job3'), ('Job4', 'End-Flow')])

In [14]:
# display using graphviz
from ctm_python_client.utils.displayDAG import DisplayDAG

# sudo apt-get install graphviz (on unix)
#  or
# brew install graphviz (for mac)
# DisplayDAG(t1_flow).display_graphviz()

## Step 6B - Display JSON

In [15]:
t1_flow.display_json()

{
    "Defaults": {
        "Application": "Naga0.3_Examples",
        "SubApplication": "Demo-OR_JOB",
        "RunAs": "ctmuser",
        "Host": "acb-rhctmv20",
        "When": {
            "Months": [
                "JAN",
                "OCT",
                "DEC"
            ],
            "MonthDays": [
                "ALL"
            ],
            "WeekDays": [
                "MON",
                "TUE",
                "WED",
                "THU",
                "FRI"
            ],
            "FromTime": "0300",
            "ToTime": "2100"
        }
    },
    "OR-JOB": {
        "Type": "Folder",
        "Start-Flow": {
            "Type": "Job:Dummy",
            "eventsToAdd": {
                "Type": "AddEvents",
                "Events": [
                    {
                        "Event": "Start-Flow-To-Job1"
                    }
                ]
            }
        },
        "Job1": {
            "Type": "Job:Dummy",
            "if-true": {
    

# Step 7 - Submit Workflow to Control-M

In [16]:
t1_flow.deploy()

[{'deployed_calendars': None,
 'deployed_connection_profiles': None,
 'deployed_drivers': None,
 'deployed_folders': ['OR-JOB'],
 'deployed_jobs': None,
 'deployment_file': 'jobs.json',
 'errors': None,
 'is_deploy_descriptor_valid': False,
 'successful_calendars_count': None,
 'successful_connection_profiles_count': 0,
 'successful_drivers_count': 0,
 'successful_folders_count': 0,
 'successful_jobs_count': 6,
 'successful_smart_folders_count': 1,
 'successful_sub_folders_count': 0,
 'warnings': None}]
Successfully submitted to Control-M
Login to None/ControlM/ and use your workflow


[{'deployed_calendars': None,
  'deployed_connection_profiles': None,
  'deployed_drivers': None,
  'deployed_folders': ['OR-JOB'],
  'deployed_jobs': None,
  'deployment_file': 'jobs.json',
  'errors': None,
  'is_deploy_descriptor_valid': False,
  'successful_calendars_count': None,
  'successful_connection_profiles_count': 0,
  'successful_drivers_count': 0,
  'successful_folders_count': 0,
  'successful_jobs_count': 6,
  'successful_smart_folders_count': 1,
  'successful_sub_folders_count': 0,
  'warnings': None}]

In [17]:
t1_flow.run()

{'monitor_page_uri': None,
 'run_id': '92d66bff-ebd8-48e9-8f38-c3db998b7b62',
 'status_uri': 'https://acb-rhctmv20.centralus.cloudapp.azure.com:8443/automation-api/run/status/92d66bff-ebd8-48e9-8f38-c3db998b7b62'}
Successfully Run job on Control-M
Login to None/ControlM/ and use your workflow


True